In [2]:
import pandas as pd
from myfun.addcol import addcolumns
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
import joblib
import os

In [33]:
weather_df = pd.read_csv(r"D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\10mins_weather_data.csv")
weather_df["DateTime"] =  pd.to_datetime(weather_df["DateTime"])
weather_df["日期"] = pd.to_datetime(weather_df["DateTime"].dt.date)
weather_df["時間"] = weather_df["DateTime"].dt.hour.astype(str).str.zfill(2)
weather_df["時段"] = weather_df["DateTime"].dt.minute.astype(str).str.zfill(2)
# weather_df = weather_df[(weather_df["DateTime"].dt.hour >= 9) & (weather_df["DateTime"].dt.hour < 17)]
weather_df["時段"]

0        00
1        10
2        20
3        30
4        40
         ..
45728    20
45729    30
45730    40
45731    50
45732    00
Name: 時段, Length: 45733, dtype: object

In [27]:
df1 = pd.read_csv(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\train_combine_groupby\\01_traingroupby.csv")
df1 = addcolumns(df1)
df1.dtypes

Serial                     object
LocationCode                int64
TimeGroup                  object
WindSpeed(m/s)            float64
Pressure(hpa)             float64
Temperature(°C)           float64
Humidity(%)               float64
Sunlight(Lux)             float64
Power(mW)                 float64
日期                 datetime64[ns]
時間                         object
時段                         object
位置                         object
dtype: object

In [34]:
weather_df.isna().sum()

StationId                         0
DateTime                          0
SunshineDuration                  0
UVIndex                           0
AirTemperature                    0
DailyExtremeHighAirTemperature    0
DailyExtremeLowAirTemperature     0
RelativeHumidity                  0
VisibilityDescription             0
StationLatitude                   0
StationLongitude                  0
StationAltitude                   0
日期                                0
時間                                0
時段                                0
dtype: int64

In [38]:
weather_df.columns

Index(['StationId', 'DateTime', 'SunshineDuration', 'UVIndex',
       'AirTemperature', 'DailyExtremeHighAirTemperature',
       'DailyExtremeLowAirTemperature', 'RelativeHumidity',
       'VisibilityDescription', 'StationLatitude', 'StationLongitude',
       'StationAltitude', '日期', '時間', '時段'],
      dtype='object')

In [37]:
weather_df.loc[weather_df["日期"]=="2024-01-06",]

,StationId,DateTime,SunshineDuration,UVIndex,AirTemperature,DailyExtremeHighAirTemperature,DailyExtremeLowAirTemperature,RelativeHumidity,VisibilityDescription,StationLatitude,StationLongitude,StationAltitude,日期,時間,時段
719,466990,2024-01-06,5.7,0,16.8,23.9,15.2,74,>30,23.975128,121.61327,16.1,2024-01-06,00,00


In [28]:
df1 = pd.read_csv(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\train_combine_groupby\\01_traingroupby.csv")
df1 = addcolumns(df1)
def combine_weather(df1:pd.DataFrame):
    weather_df = pd.read_csv(r"D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\10mins_weather_data.csv")
    weather_df["DateTime"] =  pd.to_datetime(weather_df["DateTime"])
    weather_df["日期"] = pd.to_datetime(weather_df["DateTime"].dt.date)
    weather_df["時間"] = weather_df["DateTime"].dt.hour.astype(str).str.zfill(2)
    weather_df["時段"] = weather_df["DateTime"].dt.minute.astype(str).str.zfill(2)
    
    weather_df.fillna(0,inplace=True)
    df = pd.merge(
        df1,
        weather_df,
        left_on=["日期","時間","時段"],
        right_on=["日期","時間","時段"],
        how="left",
        suffixes=["","_r"]
    )

    return(df)
comdf = combine_weather(df1)
comdf.isna().sum()
comdf.loc[comdf["UVIndex"].isna(),["日期","時間","時段"]]

,日期,時間,時段
337,2024-01-06,06,20
338,2024-01-06,06,30
339,2024-01-06,06,40
340,2024-01-06,06,50
341,2024-01-06,07,00
...,...,...,...
10268,2024-07-14,18,20
10269,2024-07-14,18,30
10270,2024-07-14,18,40
10271,2024-07-14,18,50


In [ ]:
def combine_weather(df1:pd.DataFrame):
    weather_df = pd.read_csv(r"D:\KPI報告\ml_power\station466990weather.csv")
    weather_df = weather_df.loc[:,["DateTime","TX01","GR01","RH01","UV01"]]
    weather_df["DateTime"] = weather_df["DateTime"].astype(str)
    weather_df["日期"] = pd.to_datetime(weather_df["DateTime"].str.slice(0,8))
    weather_df["時間"] = weather_df["DateTime"].str.slice(8,10)
    
    weather_df.fillna(0,inplace=True)
    df = pd.merge(
        df1,
        weather_df,
        left_on=["日期","時間"],
        right_on=["日期","時間"],
        how="left",
        suffixes=["","_r"]
    )

    return(df)
combine_weather(df1)

,Serial,LocationCode,TimeGroup,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),日期,...,SunshineDuration_r,UVIndex_r,AirTemperature_r,DailyExtremeHighAirTemperature_r,DailyExtremeLowAirTemperature_r,RelativeHumidity_r,VisibilityDescription_r,StationLatitude_r,StationLongitude_r,StationAltitude_r
0,20240101090017,17,2024-01-01 09:00:00,0.0,1024.415,19.00,65.30,5758.166,7.227,2024-01-01,...,0.0,0,18.8,19.3,18.6,64,7-10,23.975128,121.61327,16.1
1,20240101091017,17,2024-01-01 09:10:00,0.0,1024.448,19.01,65.15,5778.751,7.809,2024-01-01,...,0.0,0,18.9,19.3,18.6,64,21-30,23.975128,121.61327,16.1
2,20240101092017,17,2024-01-01 09:20:00,0.0,1024.537,19.15,64.65,7257.168,11.540,2024-01-01,...,0.0,1,18.9,19.3,18.6,64,21-30,23.975128,121.61327,16.1
3,20240101093017,17,2024-01-01 09:30:00,0.0,1024.625,19.30,64.67,7517.168,12.461,2024-01-01,...,0.0,1,19.1,19.3,18.6,65,21-30,23.975128,121.61327,16.1
4,20240101094017,17,2024-01-01 09:40:00,0.0,1024.560,19.45,64.39,7619.084,13.214,2024-01-01,...,0.0,1,19.1,19.3,18.6,64,21-30,23.975128,121.61327,16.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5843,20240712145017,17,2024-07-12 14:50:00,0.0,1004.010,36.58,64.56,11530.249,29.948,2024-07-12,...,8.8,8,31.5,32.5,26.9,77,>30,23.975128,121.61327,16.1
5844,20240712150017,17,2024-07-12 15:00:00,0.0,1003.978,36.33,63.15,11156.418,28.025,2024-07-12,...,8.9,7,31.3,32.5,26.9,76,>30,23.975128,121.61327,16.1
5845,20240712151017,17,2024-07-12 15:10:00,0.0,1004.064,36.11,65.83,13226.666,41.568,2024-07-12,...,9.1,6,31.3,32.5,26.9,78,>30,23.975128,121.61327,16.1
5846,20240712152017,17,2024-07-12 15:20:00,0.0,1004.078,36.39,65.17,12541.167,36.892,2024-07-12,...,9.3,6,31.5,32.5,26.9,75,>30,23.975128,121.61327,16.1


In [41]:
def selecttime(df:pd.DataFrame):
    df["TimeGroup"] = pd.to_datetime(df["TimeGroup"])
    df = df[(df["TimeGroup"].dt.hour >= 9) & (df["TimeGroup"].dt.hour < 17)| ((df["TimeGroup"].dt.hour == 17) & (df["TimeGroup"].dt.minute == 0))]
    return(df)
selecttime(combine_weather(df1))

,Serial,LocationCode,TimeGroup,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),日期,...,SunshineDuration_r,UVIndex_r,AirTemperature_r,DailyExtremeHighAirTemperature_r,DailyExtremeLowAirTemperature_r,RelativeHumidity_r,VisibilityDescription_r,StationLatitude_r,StationLongitude_r,StationAltitude_r
15,20240101090001,1,2024-01-01 09:00:00,1.91588,1017.906000,18.33,92.29,8395.250000,17.368000,2024-01-01,...,0.0,0.0,18.8,19.3,18.6,64.0,7-10,23.975128,121.61327,16.1
16,20240101091001,1,2024-01-01 09:10:00,0.09000,1017.965000,18.02,97.37,7046.501000,12.612000,2024-01-01,...,0.0,0.0,18.9,19.3,18.6,64.0,21-30,23.975128,121.61327,16.1
17,20240101092001,1,2024-01-01 09:20:00,0.01800,1018.163000,17.73,99.93,5919.584000,8.807000,2024-01-01,...,0.0,1.0,18.9,19.3,18.6,64.0,21-30,23.975128,121.61327,16.1
18,20240101093001,1,2024-01-01 09:30:00,0.01800,1018.205000,17.90,100.00,8838.918000,17.564000,2024-01-01,...,0.0,1.0,19.1,19.3,18.6,65.0,21-30,23.975128,121.61327,16.1
19,20240101094001,1,2024-01-01 09:40:00,0.03600,1018.131000,18.08,100.00,5774.665000,7.508000,2024-01-01,...,0.0,1.0,19.1,19.3,18.6,64.0,21-30,23.975128,121.61327,16.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10612,20240724162001,1,2024-07-24 16:20:00,0.00000,959.121000,25.10,100.00,758.085000,0.106000,2024-07-24,...,0.1,0.0,25.8,28.7,25.3,93.0,21-30,23.975128,121.61327,16.1
10613,20240724163001,1,2024-07-24 16:30:00,0.00000,959.102000,25.10,100.00,507.167000,0.054000,2024-07-24,...,0.1,0.0,26.9,28.7,25.3,89.0,16-20,23.975128,121.61327,16.1
10614,20240724164001,1,2024-07-24 16:40:00,0.00000,958.566667,25.10,100.00,369.998889,0.025556,2024-07-24,...,0.1,0.0,27.1,28.7,25.3,87.0,3-6,23.975128,121.61327,16.1
10615,20240724165001,1,2024-07-24 16:50:00,0.00000,958.926000,25.16,100.00,336.249000,0.020000,2024-07-24,...,0.1,0.0,26.9,28.7,25.3,88.0,3-6,23.975128,121.61327,16.1


In [39]:
os.cpu_count()

16

In [47]:
df1.dtypes

Serial                                    object
LocationCode                               int64
TimeGroup                         datetime64[ns]
WindSpeed(m/s)                           float64
Pressure(hpa)                            float64
Temperature(°C)                          float64
Humidity(%)                              float64
Sunlight(Lux)                            float64
Power(mW)                                float64
日期                                datetime64[ns]
時間                                        object
時段                                        object
位置                                        object
StationId                                float64
DateTime                          datetime64[ns]
SunshineDuration                         float64
UVIndex                                  float64
AirTemperature                           float64
DailyExtremeHighAirTemperature           float64
DailyExtremeLowAirTemperature            float64
RelativeHumidity    

In [ ]:
d = {}
for i in range(1,18):
    for j in range(1,18):
        if i==j:
            continue
        l1 = "0" + str(i) if i<10 else str(i)
        l2 = "0" + str(j) if j<10 else str(j)
        df1 = pd.read_csv(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\train_combine_groupby\\{l1}_traingroupby.csv")
        df2 = pd.read_csv(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\train_combine_groupby\\{l2}_traingroupby.csv")
        df1 = addcolumns(df1)
        df1 = combine_weather(df1)
        df1 = selecttime(df1)
        df2 = addcolumns(df2)
        df2 = combine_weather(df2)
        df2 = selecttime(df2)
        
        df1.dropna(inplace=True)
        df2.dropna(inplace=True)
        merge_df = pd.merge(
            df1,
            df2,
            left_on=['日期', '時間', '時段'],    # 左表連接的鍵
            right_on=['日期', '時間', '時段'], # 右表連接的鍵
            how='inner',
            suffixes=["_x","_y"]                        
        )

        X = merge_df.loc[:,["WindSpeed(m/s)_x","Pressure(hpa)_x", "Temperature(°C)_x","Humidity(%)_x","Sunlight(Lux)_x",
                            'SunshineDuration_x', 'UVIndex_x',
                            'AirTemperature_x', 'DailyExtremeHighAirTemperature_x',
                            'DailyExtremeLowAirTemperature_x', 'RelativeHumidity_x',
                            # 'VisibilityDescription_x',
                            "Power(mW)_x"]].reset_index(drop=True)
        Y = merge_df.loc[:,["Power(mW)_y"]].reset_index(drop=True)
        Y = Y.values.ravel()
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.01, random_state=42)

        model = RandomForestRegressor(
            n_jobs=12
        )
        scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
        MAE = -sum(scores)
        model.fit(X_train, y_train)
       
        model_file_name = f'{l2}_{l1}.pkl'  #l2的模型鐘用l1預測的
        model_floder_name = "train_use_full_data_update_weather_data"         #將模型相關資訊儲存在這個檔案下
        model_file_path = os.path.join(r"D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location",model_floder_name,model_file_name)
        os.makedirs(os.path.dirname(model_file_path), exist_ok=True)
        joblib.dump(model, model_file_path)
        print(model_file_name ," MAE:",MAE )
        print(f"模型已保存至: {model_file_path}")
        if l2 not in d:
            d[l2]={}
        d[l2][model_file_name] =MAE

02_01.pkl  MAE: 157.29159529658952
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_update_weather_data\02_01.pkl
03_01.pkl  MAE: 189.660406351199
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_update_weather_data\03_01.pkl
04_01.pkl  MAE: 196.24776104965508
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_update_weather_data\04_01.pkl
05_01.pkl  MAE: 263.58221856768535
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_update_weather_data\05_01.pkl
06_01.pkl  MAE: 233.06267832216113
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_update_weather_data\06_01.pkl
07_01.pkl  MAE: 170.8551267841864
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-

# 紀錄每個model的MAE並且存成json檔案

In [49]:
import json
json_output = json.dumps(d,indent=4)
output_path = f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\model_each_location\\{model_floder_name}\\d.json"
with open(output_path, "w", encoding="utf-8") as file:
    file.write(json_output)

In [50]:
with open(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\model_each_location\\{model_floder_name}\\d.json", "r", encoding="utf-8") as file:
    mae_data = json.load(file)

In [51]:
#讀入json檔案  必且將每個模型的mae由小到大依序排序
for k in mae_data.keys():
    mae_data[k] = dict(sorted(mae_data[k].items(), key=lambda item: item[1]))
mae_data

{'02': {'02_07.pkl': 131.63387368015486,
  '02_03.pkl': 148.9494026377359,
  '02_01.pkl': 157.29159529658952,
  '02_06.pkl': 164.8974798982957,
  '02_04.pkl': 178.3880294357141,
  '02_05.pkl': 236.92519867868288,
  '02_14.pkl': 250.27164768944252,
  '02_13.pkl': 271.7814698951735,
  '02_10.pkl': 296.6597617956016,
  '02_09.pkl': 364.5657067406608,
  '02_11.pkl': 486.5527985457143,
  '02_08.pkl': 490.6013231639061,
  '02_12.pkl': 500.5751417951683,
  '02_16.pkl': 602.7255868558245,
  '02_15.pkl': 653.4253487692202,
  '02_17.pkl': 707.270496562365},
 '03': {'03_02.pkl': 161.0008668940303,
  '03_04.pkl': 189.5358006131556,
  '03_01.pkl': 189.660406351199,
  '03_06.pkl': 227.4046924454917,
  '03_07.pkl': 234.48687958859145,
  '03_05.pkl': 264.339052888098,
  '03_10.pkl': 314.98827932300026,
  '03_13.pkl': 364.1884834303437,
  '03_14.pkl': 379.6113030335456,
  '03_11.pkl': 466.2711881442685,
  '03_09.pkl': 476.82371276837563,
  '03_12.pkl': 566.1586075627325,
  '03_08.pkl': 612.192990029394

In [ ]:
#將不同觀測站的資料load近來並放在字典裡
df_dict = {}
for i in range(1,18):
    location = "0" + str(i) if i<10 else str(i)
    df = pd.read_csv(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\train_combine_groupby\\{location}_traingroupby.csv")
    df = addcolumns(df)
    df = combine_weather(df)
    df_dict[location] = df

In [53]:
#將不同模型load近來並放在字典裡
model_dict = {}
for k1 in mae_data.keys():
    for model_name in mae_data[k1].keys():
        model_file_path = os.path.join(r"D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location",model_floder_name,model_name)
        model_dict[model_name] = joblib.load(model_file_path)

In [ ]:
#將檔案讀進來  新增日期時間位置等欄位以便我們找資料對應
DataName = "./36_TestSet_SubmissionTemplate/upload(no answer).csv"
ans = pd.read_csv(DataName, encoding='utf-8')
ans["序號"] = ans["序號"].astype(str)
ans["日期"] = pd.to_datetime(ans["序號"].str.slice(0,8))
ans["時間"] =ans["序號"].str.slice(8,10)
ans["時段"] =ans["序號"].str.slice(10,12)
ans["位置"] =ans["序號"].str.slice(12,14)
ans["模型順位"] = None
ans["使用模型MAE"] = None
for i in range(len(ans)):
# for i in range(3):  
    date,time,timecount,ans_location = ans.loc[i,"日期"],ans.loc[i,"時間"],ans.loc[i,"時段"],ans.loc[i,"位置"]  #找出答案該個row的時期時間時段以及位置
    mae_table = mae_data[ans_location]   #找出這個位置的所有模型(來自另外16個模型) 並且已按照MAE順序排
    j = 1
    for model_name in mae_table.keys():  #從MAE最小的模型(觀測站)去找有沒有該觀測站的資料  如果有就用該資料跟模型來進行predict
        X_location = model_name[3:5]     # X_locatioan是訓練集模型的地點
        df_sametimedata = df_dict[X_location]
        # df_sametimedata = addcolumns(df_sametimedata)
        # df_sametimedata = combine_weather(df_sametimedata)
        search_df = df_sametimedata.loc[(df_sametimedata["日期"]==date) & (df_sametimedata["時間"]==time) & (df_sametimedata["時段"]==timecount),]
        if len(search_df)==1:  #如果有找資料 就用這個模型predict 並且break迴圈  沒有就繼續往MAE較大的模型找 找到都沒有則答案欄位就保持NA
            X_test = search_df.loc[:,["WindSpeed(m/s)","Pressure(hpa)", "Temperature(°C)","Humidity(%)","Sunlight(Lux)",
                            'SunshineDuration', 'UVIndex',
                            'AirTemperature', 'DailyExtremeHighAirTemperature',
                            'DailyExtremeLowAirTemperature', 'RelativeHumidity',
                            'Power(mW)']]  
            X_test = X_test.rename(columns=lambda col: f"{col}_x")  #將每個變數都加上_x以符合train那邊的形式
            model = model_dict[model_name]
            predictions = model.predict(X_test)
            ans.loc[i,"答案"] = predictions[0]
            ans.loc[i,"模型順位"] = j
            ans.loc[i,"使用模型MAE"] = mae_table[model_name]
            break
        j = j+1 
    if i %100==0:
        print(i)

            

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500


In [56]:
ans_nona = ans.loc[~ans["答案"].isna(),].reset_index(drop=True)
ans_na = ans.loc[ans["答案"].isna(),].reset_index(drop=True)

In [57]:
ans_nona.loc[ans["使用模型MAE"]>=700,]

,序號,答案,日期,時間,時段,位置,模型順位,使用模型MAE


In [ ]:
model_lastday_floder_name = "train_lastday_gridsearch_usealldata"
with open(f"./model_each_location/{model_lastday_floder_name}/d.json", "r", encoding="utf-8") as file:
    mae_lastday_data = json.load(file)

In [66]:
mae_lastday_data

{'01_weather.pkl': 592.6372671878407,
 '02_weather.pkl': 689.1069618931544,
 '03_weather.pkl': 752.4351211995079,
 '04_weather.pkl': 627.3513850151596,
 '05_weather.pkl': 776.0900688386986,
 '06_weather.pkl': 854.3954272525956,
 '07_weather.pkl': 810.5925003147652,
 '08_weather.pkl': 497.54657773877,
 '09_weather.pkl': 517.314216845766,
 '10_weather.pkl': 609.0788206888952,
 '11_weather.pkl': 273.7172400610061,
 '12_weather.pkl': 508.5767234536611,
 '13_weather.pkl': 539.8340001971161,
 '14_weather.pkl': 668.1270343345218,
 '15_weather.pkl': 636.5045183516974,
 '16_weather.pkl': 568.1460919654046,
 '17_weather.pkl': 513.2776679465638}

In [67]:
model_lastday_dict = {}
for k1 in mae_lastday_data.keys():
    model_name = k1 
    model_file_path = os.path.join(r"D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location",model_lastday_floder_name,model_name)
    model_lastday_dict[k1] = joblib.load(model_file_path)

In [75]:
#當答案欄位有空格  就會去找MAE最小的模型  並且用前一天的資料去predict 沒前一天的資料就找更前一天
for i in range(len(ans_na)):
# for i in range(3):  
    if i%10==0:
        print(i)
    time,timecount,ans_location = ans_na.loc[i,"時間"],ans_na.loc[i,"時段"],ans_na.loc[i,"位置"]
    # mae_table = mae_data[ans_location]
    # model_name = list(mae_table.keys())[0]  
    # X_location = model_name[3:5]
    df_sametimedata = df_dict[ans_location]
    df_sametimedata = addcolumns(df_sametimedata)
    df_sametimedata = combine_weather(df_sametimedata)
    days = 1
    while True:
        date = ans_na.loc[i,"日期"] - pd.Timedelta(days=days)
        search_df = df_sametimedata.loc[(df_sametimedata["日期"]==date) & (df_sametimedata["時間"]==time) & (df_sametimedata["時段"]==timecount),]
        if len(search_df)==1:
            X_test = search_df.loc[:,[
                            'SunshineDuration', 'UVIndex',
                            'AirTemperature', 'DailyExtremeHighAirTemperature',
                            'DailyExtremeLowAirTemperature', 'RelativeHumidity']]  
            # X_test = X_test.rename(columns=lambda col: f"{col}_x")
            model = model_lastday_dict[f"{ans_location}_weather.pkl"]
            predictions = model.predict(X_test)
            ans_na.loc[i,"答案"] = predictions[0]
            break
        days = days + 1

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


In [76]:
res = pd.concat([ans_nona,ans_na],axis=0).reset_index(drop=True)
res.loc[:,["序號","答案"]].to_csv("1128ans3_alldata_udate_weather.csv",index=False)

In [11]:
merge_df.loc[:,["Temperature(°C)_x","Humidity(%)_x","Sunlight(Lux)_x","Power(mW)_x","Temperature(°C)_y","Humidity(%)_y","Sunlight(Lux)_y","Power(mW)_y"]].corr()

,Temperature(°C)_x,Humidity(%)_x,Sunlight(Lux)_x,Power(mW)_x,Temperature(°C)_y,Humidity(%)_y,Sunlight(Lux)_y,Power(mW)_y
Temperature(°C)_x,1.000000,-0.865882,0.713088,0.663425,0.985160,-0.743495,0.668910,0.649906
Humidity(%)_x,-0.865882,1.000000,-0.722009,-0.683149,-0.844816,0.902799,-0.704469,-0.677605
Sunlight(Lux)_x,0.713088,-0.722009,1.000000,0.983972,0.726162,-0.761528,0.982015,0.979144
Power(mW)_x,0.663425,-0.683149,0.983972,1.000000,0.678282,-0.720957,0.973662,0.991635
Temperature(°C)_y,0.985160,-0.844816,0.726162,0.678282,1.000000,-0.744065,0.688686,0.668247
Humidity(%)_y,-0.743495,0.902799,-0.761528,-0.720957,-0.744065,1.000000,-0.740842,-0.714182
Sunlight(Lux)_y,0.668910,-0.704469,0.982015,0.973662,0.688686,-0.740842,1.000000,0.983198
Power(mW)_y,0.649906,-0.677605,0.979144,0.991635,0.668247,-0.714182,0.983198,1.000000
